In [9]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, Flatten, MaxPooling1D, Dropout
from sklearn.metrics import recall_score, precision_score

# Load the data

In [10]:
data = pd.read_csv('god-class-2020+2019+2018.csv')
data = data[['WMCNAMM_type', 'LOC_type','is_god_class']]

# Normalize the data

In [11]:
scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(data.drop(columns='is_god_class'))
X = data_scaled.reshape(data_scaled.shape[0], data_scaled.shape[1], 1)
y = data['is_god_class'].values

# Split data

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Model Building

In [13]:
model = Sequential()

model.add(Conv1D(filters=32, kernel_size=2, activation='relu', input_shape=(X_train.shape[1], 1)))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_1 (Conv1D)           (None, 1, 32)             96        
                                                                 
 flatten_1 (Flatten)         (None, 32)                0         
                                                                 
 dense_3 (Dense)             (None, 128)               4224      
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 
 dense_4 (Dense)             (None, 64)                8256      
                                                                 
 dense_5 (Dense)             (None, 1)                 65        
                                                                 
Total params: 12,641
Trainable params: 12,641
Non-trai

# Train the model

In [14]:
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/10
8/8 [==============================] - 2s 92ms/step - loss: 0.6843 - accuracy: 0.6419 - val_loss: 0.6709 - val_accuracy: 0.5690
Epoch 2/10
8/8 [==============================] - 0s 33ms/step - loss: 0.6672 - accuracy: 0.5808 - val_loss: 0.6515 - val_accuracy: 0.5690
Epoch 3/10
8/8 [==============================] - 0s 14ms/step - loss: 0.6491 - accuracy: 0.5764 - val_loss: 0.6370 - val_accuracy: 0.5690
Epoch 4/10
8/8 [==============================] - 0s 14ms/step - loss: 0.6383 - accuracy: 0.5764 - val_loss: 0.6176 - val_accuracy: 0.5690
Epoch 5/10
8/8 [==============================] - 0s 16ms/step - loss: 0.6247 - accuracy: 0.6419 - val_loss: 0.5984 - val_accuracy: 0.8103
Epoch 6/10
8/8 [==============================] - 0s 14ms/step - loss: 0.6083 - accuracy: 0.7904 - val_loss: 0.5812 - val_accuracy: 0.8103
Epoch 7/10
8/8 [==============================] - 0s 14ms/step - loss: 0.5990 - accuracy: 0.8079 - val_loss: 0.5676 - val_accuracy: 0.8103
Epoch 8/10
8/8 [===========

# Evaluation

In [15]:
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=2)
print("\nTest accuracy:", test_acc)

2/2 - 0s - loss: 0.5240 - accuracy: 0.8103 - 53ms/epoch - 26ms/step

Test accuracy: 0.8103448152542114


In [16]:
from sklearn.metrics import recall_score, precision_score
from sklearn.metrics import f1_score
y_pred_probs = model.predict(X_test)
y_pred = (y_pred_probs > 0.5).astype(int).flatten()

recall = recall_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f"Recall: {recall}")
print(f"Precision: {precision}")
print(f"F1 Score: {f1}")

2/2 [==============================] - 0s 8ms/step
Recall: 0.8484848484848485
Precision: 0.8235294117647058
F1 Score: 0.8358208955223881


In [ ]:
import joblib
model.save('gc_model.h5')
joblib.dump(scaler, 'gc_scalar.pkl')